In [24]:
# Импорт библиотек для работы с COM-портом и многопоточностью

import numpy as np
import threading 
import time 
import serial
import serial.tools.list_ports

In [25]:
# Получение списка доступных COM-портов

ports = serial.tools.list_ports.comports()
for port in ports:
    print(port.device)

COM3
COM4
COM5


In [12]:
# Установка соединения с портом

port = "COM1" # Выбор порта
baudrate = 9600 # Скорость порта

ser = serial.Serial(port, baudrate = baudrate) # Инициализация порта

ser.close() # Закрыть порт

SerialException: could not open port 'COM1': FileNotFoundError(2, 'Не удается найти указанный файл.', None, 2)

In [26]:
# Создаем два параллельных потока - получение данных о светодиоде от STM32 и запрос о корректной работе STM32

def request_byte(): # Функция потока "запрос-ответ"
    try:
        ser = serial.Serial(port, baudrate = baudrate) # Открыли порт
        ser.write(0x32) # Записали в порт 16-ричное число 0х34
    except serial.SerialException as se: # Обработка исключения - в случае ошибки
        print("Serial port error:", str(se))
    finally:
        if ser.is_open:
            ser.close() # Закрыть порт
    time.sleep(0.2) # Ждем 0.2 секунды
    flg
    try:
        for _ in range(5): # Пытаемся 5 раз считать состояние порта
            ser = serial.Serial(port, baudrate = baudrate)
            rec = ser.read() # Считываем принятые данные из порта
            if rec == 0x32: # Если принятый бит соответствует запросу, говорим, что STM на связи, если не соответствует - нет связи
                flg = 1
                print("STM на связи")
                break
            else: flg = 0
        if (flg == 0): print("Нет связи с STM")
    except serial.SerialException as er: # Обработка исключения - в случае ошибки
        print("Serial port error:", str(er))
    finally:
        if ser.is_open:
            ser.close() # Закрыть порт

def receive_byte(): # Функция потока "прием данных о состоянии светодиода"
    try:
        ser = serial.Serial(port, baudrate = baudrate)
        while True:
            rec = ser.read() # Считываем принятые данные из порта
            if rec == 0x12: # Если принятый бит = 0х12, светодиод горит, если 0х22 - выключен
                print("Светодиод горит")
            elif rec == 0x22:
                print("Светодиод выключен")
    except serial.SerialException as se:
        print("Serial port error:", str(se))
    finally:
        if ser.is_open:
            ser.close() # Закрыть порт

if __name__ == '__main__': # Включаем двухпоточную работу - одновременно выполняются два процесса - запросов связи и приема данных о состоянии светодиода
    thread1 = threading.Thread(target = request_byte)
    thread2 = threading.Thread(target = receive_byte)
    thread1.start()
    thread2.start()


Exception in thread Thread-316 (request_byte):
Traceback (most recent call last):
  File "C:\Users\Alexander\AppData\Local\Temp\ipykernel_3288\3409999061.py", line 5, in request_byte
  File "C:\Users\Alexander\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\serial\serialwin32.py", line 33, in __init__
    super(Serial, self).__init__(*args, **kwargs)
  File "C:\Users\Alexander\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\serial\serialutil.py", line 222, in __init__
    self.port = port
    ^^^^^^^^^
  File "C:\Users\Alexander\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\serial\serialutil.py", line 268, in port
    raise ValueError('"port" must be None or a string, not {}'.format(type(port)))
ValueError: "port" must be None or a string, not <class 'serial.t

    _threading_Thread_run(self)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Alexander\AppData\Local\Temp\ipykernel_3288\3409999061.py", line 10, in request_byte
UnboundLocalError: cannot access local variable 'ser' where it is not associated with a value
  File "C:\Users\Alexander\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\serial\serialwin32.py", line 33, in __init__
    super(Serial, self).__init__(*args, **kwargs)
  File "C:\Users\Alexander\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\serial\serialutil.py", line 222, in __init__
    self.port = port
    ^^^^^^^^^
  File "C:\Users\Alexander\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCa